# Week 3 Part 1 
# Explore and cluster the neighborhoods in Toronto

In [1]:
import pandas as pd

In [5]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [6]:
!conda install -c conda-forge folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.10.0              |             py_0          59 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         882 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.10.0-py_0  conda-forge
    vincent: 0.4.4-py_1   conda-forge


altair-3.2.0         | 770 KB    | ##################################### |

In [7]:
from geopy.geocoders import Nominatim
import requests
import folium
import re
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## Reading Combined Postcode Latitude Longitude Data from Part 2

In [12]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_1460c3a0a9f94fc497ad0bc336243c7b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='KgOJIy3jyBvDMGZwJNhTG4R1YaruImrPfEBTXGBOGey8',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_1460c3a0a9f94fc497ad0bc336243c7b.get_object(Bucket='courseracapstoneproject-donotdelete-pr-ib2rqrpouoki4y',Key='combinedPostcodeLatLongData.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

Toronto_df = pd.read_csv(body)
Toronto_df.head()



,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [13]:
#Toronto_df = pd.read_csv('combinedPostcodeLatLongData.csv')
#Toronto_df.head()

# Filtering the Borough having word 'Toronto' in its name

In [14]:
BoroughString='Toronto'
BoroughAsToronto_df = Toronto_df[Toronto_df['Borough'].str.contains(BoroughString)]
BoroughAsToronto_df=BoroughAsToronto_df.reset_index(drop=True)
BoroughAsToronto_df.shape

(38, 5)

In [15]:
BoroughAsToronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [16]:
address = 'Toronto City, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.7189883, -79.44157.


In [17]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [18]:
for lat, lng, borough, neighborhood in zip(BoroughAsToronto_df['Latitude'], 
BoroughAsToronto_df['Longitude'], BoroughAsToronto_df['Borough'], BoroughAsToronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_Toronto)
map_Toronto

In [19]:
BoroughAsToronto_df.shape

(38, 5)

In [56]:
CLIENT_ID = 'Your own Client id here' 
CLIENT_SECRET = 'Your own client secret here'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Your own Client id here
CLIENT_SECRET:Your own client secret here


In [21]:
pos=1
print(BoroughAsToronto_df.loc[pos,'Neighbourhood'])
neighborhood_name = BoroughAsToronto_df.loc[pos, 'Neighbourhood'] # neighborhood name
neighborhood_latitude = BoroughAsToronto_df.loc[pos, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = BoroughAsToronto_df.loc[pos, 'Longitude'] # neighborhood longitude value
print('Latitude and longitude values of \'{}\' are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Ryerson,Garden District
Latitude and longitude values of 'Ryerson,Garden District' are 43.6571618, -79.37893709999999.


In [22]:

limit = 100 # top 100 venues in given Neighbourhood within radius(500)
radius = 500 # radius is in meters
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude,neighborhood_longitude, radius, limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ZBJYO13FKGEOV24ZQTABW3C4Q2ZUE1MYTOTRABYH3FQWHHLW&client_secret=PTA1YB2SNTJD5K1LJ11FB3VWEW1RAXT1GHSD0BMAHPCLQDYV&v=20180605&ll=43.6571618,-79.37893709999999&radius=500&limit=100'

## Call to (foursquare API) for Venue Data

In [23]:
results = requests.get(url).json()
#results

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
                
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']
#print(venues)

nearby_venues = json_normalize(venues) # flatten JSON into dataframe
nearby_venues.head(3)

,reasons.count,reasons.items,referralId,venue.categories,venue.events.count,venue.events.summary,venue.id,venue.location.address,venue.location.cc,venue.location.city,...,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.neighborhood,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-57eda381498ebe0e6ef40972-0,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",NaN,NaN,57eda381498ebe0e6ef40972,220 Yonge St,CA,Toronto,...,"[{'label': 'display', 'lat': 43.65591027779457...",43.655910,-79.380641,Downtown Toronto,M5B 2H1,ON,UNIQLO ユニクロ,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-55a9bbf9498e00ffd7f4c71f-1,"[{'id': '4bf58dd8d48988d153941735', 'name': 'B...",NaN,NaN,55a9bbf9498e00ffd7f4c71f,74 Dundas St E,CA,Toronto,...,"[{'label': 'display', 'lat': 43.65626526440901...",43.656265,-79.378343,NaN,NaN,ON,Burrito Boyz,0,[],NaN
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5615b6c4498e3c32c67ad78f-2,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",NaN,NaN,5615b6c4498e3c32c67ad78f,"10 Dundas Street East, #124",CA,Toronto,...,"[{'label': 'display', 'lat': 43.656518, 'lng':...",43.656518,-79.380015,NaN,M5B 2G9,ON,Blaze Pizza,0,[],NaN


In [26]:

# Filtering the columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
nearby_venues.head(3)


,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,UNIQLO ユニクロ,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",43.655910,-79.380641
1,Burrito Boyz,"[{'id': '4bf58dd8d48988d153941735', 'name': 'B...",43.656265,-79.378343
2,Blaze Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",43.656518,-79.380015


In [27]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.head(3)
#print(nearby_venues)

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,UNIQLO ユニクロ,Clothing Store,43.655910,-79.380641
1,Burrito Boyz,Burrito Place,43.656265,-79.378343
2,Blaze Pizza,Pizza Place,43.656518,-79.380015


In [28]:
# clean columns
# Keeping the last string and discarding all other strings after spliting the column names
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head(3)

,name,categories,lat,lng
0,UNIQLO ユニクロ,Clothing Store,43.655910,-79.380641
1,Burrito Boyz,Burrito Place,43.656265,-79.378343
2,Blaze Pizza,Pizza Place,43.656518,-79.380015


# Following are the venues returned by Foursquare

In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    print('Retrieving Venues of Neighbourhood = ')
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,limit)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(name,lat,lng,v['venue']['name'],
                                          v['venue']['location']['lat'],
                                          v['venue']['location']['lng'],
                                          v['venue']['categories'][0]['name']) for v in results])
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
                
        nearby_venues.columns = ['Neighbourhood','Neighbourhood Latitude','Neighbourhood Longitude','Venue','Venue Latitude',
        'Venue Longitude','Venue Category']
        
    return(nearby_venues)


In [31]:

# Calling foursquare API to retrieve all vanues for each BoroughAsToronto_df Neighbourhood by providing the 
# name, latitude & longitude of each Neighbourhood

BoroughAsToronto_venues = getNearbyVenues(names=BoroughAsToronto_df['Neighbourhood'],latitudes=BoroughAsToronto_df['Latitude'],longitudes=BoroughAsToronto_df['Longitude'])
                                          

Retrieving Venues of Neighbourhood = 
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town


In [32]:
# Size of the resulting dataframe
print(BoroughAsToronto_venues.shape)
BoroughAsToronto_venues.head() # Retrieved venues data against each Neighbourhood

(1699, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [33]:
BoroughAsToronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton,Exhibition Place,Parkdale Village",21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",45,45,45,45,45,45
Central Bay Street,84,84,84,84,84,84
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15


In [34]:
print('There are {} uniques categories.'.format(len(BoroughAsToronto_venues['Venue Category'].unique())))

There are 239 uniques categories.


## Analysis of Each Neighborhood

In [35]:
# one hot encoding
BoroughAsToronto_onehot = pd.get_dummies(BoroughAsToronto_venues[['Venue Category']], prefix="", prefix_sep="")
# BoroughAsToronto_onehot.shape # see the new columns are equal to unique venue categories
BoroughAsToronto_onehot


# add neighborhood column back to dataframe
BoroughAsToronto_onehot['Neighbourhood'] = BoroughAsToronto_venues['Neighbourhood']
BoroughAsToronto_onehot


# move neighborhood column as a first column
fixed_columns = [BoroughAsToronto_onehot.columns[-1]] + list(BoroughAsToronto_onehot.columns[:-1])
BoroughAsToronto_onehot = BoroughAsToronto_onehot[fixed_columns]
BoroughAsToronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
BoroughAsToronto_onehot.shape # 239 unique venue category columns plus one Neighborhood column = 240 columns

(1699, 240)

In [37]:
BoroughAsToronto_onehot_grouped = BoroughAsToronto_onehot.groupby('Neighbourhood').mean().reset_index()
BoroughAsToronto_onehot_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.000000,0.00000,0.00,0.010000,0.000000,0.000000,0.000000,0.010000,0.01,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.052632
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,0.000000,...,0.000000,0.00000,0.00,0.011905,0.000000,0.011905,0.000000,0.011905,0.00,0.011905
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.060000,0.000000,0.000000,0.040000,0.010000,0.00,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Church and Wellesley,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,0.000000,...,0.000000,0.00000,0.00,0.000000,0.011905,0.000000,0.011905,0.000000,0.00,0.011905


In [38]:
BoroughAsToronto_onehot_grouped.shape

(38, 240)

In [39]:
num_top_venues = 5
for hood in BoroughAsToronto_onehot_grouped['Neighbourhood']:
    print("----"+hood+"----")
    
    #temp1 = BoroughAsToronto_onehot_grouped[BoroughAsToronto_onehot_grouped['Neighbourhood'] == hood].T.reset_index()
    
    temp = BoroughAsToronto_onehot_grouped[BoroughAsToronto_onehot_grouped['Neighbourhood'] == hood].T.reset_index()
    #print(temp)
    temp.columns = ['venue','freq'] # changing the name of columns
    #print(temp)
    temp = temp.iloc[1:]
    #print(temp)
    temp['freq'] = temp['freq'].astype(float)
    #print(temp)
    temp = temp.round({'freq': 2})
    #print(temp)    
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.08
1             Café  0.05
2              Bar  0.04
3  Thai Restaurant  0.04
4       Steakhouse  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2     Cheese Shop  0.04
3  Farmers Market  0.04
4            Café  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0     Coffee Shop  0.10
1            Café  0.10
2  Breakfast Spot  0.10
3      Restaurant  0.05
4    Climbing Gym  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2       Auto Workshop  0.05
3                 Spa  0.05
4         Pizza Place  0.05


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0      Airport Lounge  0.14
1     Airport Service  0.14
2    Airport Terminal  0.14
3    

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]    

In [41]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd'] # for 1st, 2nd & 3rd

# create columns according to number of top venues
columns = ['Neighbourhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
    
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)

neighborhoods_venues_sorted['Neighbourhood'] = BoroughAsToronto_onehot_grouped['Neighbourhood']
#print(neighborhoods_venues_sorted)

for ind in np.arange(BoroughAsToronto_onehot_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(BoroughAsToronto_onehot_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Breakfast Spot,Hotel,Gym,Restaurant,Asian Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Seafood Restaurant,Beer Bar,Café,Cheese Shop,Farmers Market,Bakery,Concert Hall
2,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Coffee Shop,Café,Italian Restaurant,Pet Store,Climbing Gym,Restaurant,Caribbean Restaurant,Burrito Place,Stadium
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Garden Center,Farmers Market,Smoke Shop,Fast Food Restaurant,Brewery,Park,Burrito Place,Spa
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Plane,Airport,Airport Food Court,Airport Gate,Coffee Shop,Harbor / Marina,Sculpture Garden


## K-Means Clustering

In [42]:
# set number of clusters
kclusters = 5
BoroughAsToronto_onehot_grouped_clustering = BoroughAsToronto_onehot_grouped.drop('Neighbourhood', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(BoroughAsToronto_onehot_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#neighborhoods_venues_sorted

BToronto_merged = BoroughAsToronto_df
#BToronto_merged.head(3)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
BToronto_merged = BToronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
BToronto_merged.head() # check the last columns!


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Café,Pub,Park,Bakery,Mexican Restaurant,Breakfast Spot,Theater,Restaurant,Event Space
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Theater,Restaurant,Ramen Restaurant,Bubble Tea Shop,Plaza
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Hotel,Italian Restaurant,Restaurant,Coffee Shop,Café,Bakery,Gastropub,Breakfast Spot,Clothing Store,Cosmetics Shop
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Neighborhood,Pub,Health Food Store,Coffee Shop,Trail,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Steakhouse,Seafood Restaurant,Beer Bar,Café,Cheese Shop,Farmers Market,Bakery,Concert Hall


In [43]:
BToronto_merged.shape

(38, 16)

## Visualize

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(BToronto_merged['Latitude'], BToronto_merged['Longitude'],BToronto_merged['Neighbourhood'], BToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters


['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


## Clusters are seen seperately as:

## It can be observed following that the same colored Borough clusters we see in visualization have similar properties as they have mostly same popular venues in each of them in top 10

In [50]:
BToronto_merged.loc[BToronto_merged['Cluster Labels'] == 0, BToronto_merged.columns[[2] + list(range(6, BToronto_merged.shape[1]))]].head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Harbourfront,Regent Park",Coffee Shop,Café,Pub,Park,Bakery,Mexican Restaurant,Breakfast Spot,Theater,Restaurant,Event Space
1,"Ryerson,Garden District",Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Theater,Restaurant,Ramen Restaurant,Bubble Tea Shop,Plaza
2,St. James Town,Hotel,Italian Restaurant,Restaurant,Coffee Shop,Café,Bakery,Gastropub,Breakfast Spot,Clothing Store,Cosmetics Shop
4,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Seafood Restaurant,Beer Bar,Café,Cheese Shop,Farmers Market,Bakery,Concert Hall
5,Central Bay Street,Coffee Shop,Ice Cream Shop,Italian Restaurant,Sandwich Place,Burger Joint,Café,Bakery,Bubble Tea Shop,Salad Place,Restaurant


In [51]:
BToronto_merged.loc[BToronto_merged['Cluster Labels'] == 1, BToronto_merged.columns[[2] + list(range(6, BToronto_merged.shape[1]))]].head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Roselawn,Garden,Yoga Studio,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [52]:
BToronto_merged.loc[BToronto_merged['Cluster Labels'] == 2, BToronto_merged.columns[[2] + list(range(6, BToronto_merged.shape[1]))]].head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Lawrence Park,Park,Bus Line,Dim Sum Restaurant,Swim School,Yoga Studio,Donut Shop,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market


In [53]:
BToronto_merged.loc[BToronto_merged['Cluster Labels'] == 3, BToronto_merged.columns[[2] + list(range(6, BToronto_merged.shape[1]))]].head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,"Forest Hill North,Forest Hill West",Trail,Sushi Restaurant,Park,Jewelry Store,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Yoga Studio
28,"Moore Park,Summerhill East",Playground,Restaurant,Gym,Park,Electronics Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Event Space
32,Rosedale,Park,Playground,Building,Trail,Doner Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [54]:
BToronto_merged.loc[BToronto_merged['Cluster Labels'] == 4, BToronto_merged.columns[[2] + list(range(6, BToronto_merged.shape[1]))]].head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,The Beaches,Neighborhood,Pub,Health Food Store,Coffee Shop,Trail,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
